In [93]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [94]:
energy=pd.read_csv("energy.csv")

In [95]:
energy.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,11-01-2016 17:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.60,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,11-01-2016 17:10,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.48,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,11-01-2016 17:20,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.37,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,11-01-2016 17:30,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.25,733.8,92.0,6.000000,51.500000,5.0,45.410390,45.410390
4,11-01-2016 17:40,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.13,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [96]:
x=energy['T2'].to_numpy()
y=energy['T6'].to_numpy()
#y=y.reshape(-1, 1)

In [97]:
x=x.reshape(-1, 1)
y=y.reshape(-1, 1)

In [98]:
LG=LinearRegression()
LGmodel=LG.fit(x,y)
y_pred=LGmodel.predict(x)

In [99]:
y = pd.DataFrame(y)
y_pred=pd.DataFrame(y_pred)

In [45]:
y_pred

,0
0,5.371656
1,5.371656
2,5.371656
3,5.371656
4,5.371656
...,...
19730,20.257317
19731,19.954708
19732,19.675622
19733,19.198187


In [100]:
r2_score = r2_score(y,y_pred)


In [101]:
r2_score

0.641899083081307

In [102]:
nenergy=energy.drop(labels=['date', 'lights'],axis=1)


In [103]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normal_nenergy = pd.DataFrame(scaler.fit_transform(nenergy), columns=nenergy.columns)

In [104]:
X=normal_nenergy.drop(labels=['Appliances'],axis=1)
Y=normal_nenergy['Appliances']

In [105]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.30,random_state=42)

In [106]:
LRG=LinearRegression()
LRGmodel=LRG.fit(X_train,Y_train)
Y_pred=LRGmodel.predict(X_test)

In [107]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(Y_test, Y_pred)
round(mae, 2)

0.05

In [108]:
import numpy as np
rss = np.sum(np.square(Y_test - Y_pred))
round(rss, 2)

45.35

In [110]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(Y_test, Y_pred))
round(rmse, 3)

0.088

In [111]:
from sklearn.metrics import r2_score
r2_score = r2_score(Y_test, Y_pred)
round(r2_score, 2)

0.15

In [112]:
#comparing the effects of regularisation
def get_weights_df(model, feat, col_name):
#this function returns the weight of every feature
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['Features', col_name]
    weights_df[col_name].round(3)
    return weights_df
linear_model_weights = get_weights_df(LRGmodel, X_train, 'Linear_Model_Weight')

In [85]:
linear_model_weights

,Features,Linear_Model_Weight
0,RH_2,-0.456662
1,T_out,-0.321969
2,T2,-0.236198
3,T9,-0.189977
4,RH_8,-0.157585
5,RH_out,-0.077748
6,RH_7,-0.044620
7,RH_9,-0.039808
8,T5,-0.015645
9,T1,-0.003242


In [113]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=0.4)
ridge_reg.fit(X_train, Y_train)
Y_pred=ridge_reg.predict(X_test)
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(Y_test, Y_pred))
round(rmse, 3)

0.088

In [114]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(X_train, Y_train)
Y_pred=lasso_reg.predict(X_test)
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(Y_test, Y_pred))
round(rmse, 3)

0.094

In [115]:
#comparing the effects of regularisation
def get_weights_df(model, feat, col_name):
#this function returns the weight of every feature
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['Features', col_name]
    weights_df[col_name].round(3)
    return weights_df
linear_model_weights = get_weights_df(LRGmodel, X_train, 'Linear_Model_Weight')

In [116]:
linear_model_weights

,Features,Linear_Model_Weight
0,RH_2,-0.456662
1,T_out,-0.321969
2,T2,-0.236198
3,T9,-0.189977
4,RH_8,-0.157585
5,RH_out,-0.077748
6,RH_7,-0.044620
7,RH_9,-0.039808
8,T5,-0.015645
9,T1,-0.003242
